### 1. Setting up functions and initializing 

In [ ]:
print sc.version
print os.getcwd()
import time, re, datetime, time, os
import numpy as np
import pandas as pd

def isdigit(text):
    try:
        float(text)
        return True
    except ValueError:
        return False

extractTimeStamp = re.compile('^\w{3}\s\w{3}\s\d{2}\s\d{2}:\d{2}:\d{2}\s\d{4}')

def str2dateTime(sDate):
    return datetime.datetime.strptime(sDate, '%a %b %d %H:%M:%S %Y')

def str2utc(sDate):
    pattern = '%a %b %d %H:%M:%S %Y'
    return time.mktime(time.strptime(sDate, pattern))

In [3]:
t1 = time.time()
rdd = sc.textFile('ensco5006/*.log',100)

rddtotal = rdd.count()
print time.time()-t1


33.4264008999


### 2. All combinations including alarms and events that potentially not needed for analysis

#### when result is empty, it means the regex combinations are enough to find all the alamrs in a cell

In [108]:
rddparsed1 = (rdd.filter(lambda line: '>' in line)
              .filter(lambda line: '->' not in line)
              .filter(lambda line: re.compile('(\D+) is  > (\d+)psi; @ (\d+)psi is in a NORMAL state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('(\D+) >(\d+)psi; @ (\d+)psi is in a NORMAL state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('(\D+) > (\d+)psi; @ (\d+)psi is in a NORMAL state, Tagname = (\D+)').search(line) is None)
              .take(10)
              )

print rddparsed1


[]


In [103]:
rddparsed1 = (rdd.filter(lambda line: '<' in line)
              .filter(lambda line: re.compile('(\D+) is  < (\d+)psi; @ (\d+)psi is in (an|a) ALARM state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('(\D+) < (\d+)psi; @ (\d+)psi is in (an|a) ALARM state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('(\D+) < (\d+)psi; @ -(\d+)psi is in (an|a) ALARM state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('(\D+) < (\d+) is in (a|an) (NORMAL|ALARM) state(;|,) (Tagname|TagName) = ').search(line) is None)
              .filter(lambda line: re.compile('(\D+) < (\d+) is in (NORMAL|ALARM) state; (Tagname|TagName) = ').search(line) is None)
              .take(10)
              )

print rddparsed1

[]


In [90]:
rddparsed1 = (rdd.filter(lambda line: '=' in line)
             .filter(lambda line: '<' not in line)
             .filter(lambda line: '>' not in line)
              .filter(lambda line: re.compile('is in an ALARM state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('is in a NORMAL state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('is Not in an Alarm state, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('is Not in Alarm State, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('is in Alarm State, Tagname = (\D+)').search(line) is None)
              .filter(lambda line: re.compile('is  Not in Alarm State, Tagname = (\D+)').search(line) is None)
#             .filter(lambda line: re.compile('(\D+) CC Plc Receive data has not changed for 45 sec is in an ALARM state, Tagname = (\D+)').search(line) is None)
#             .filter(lambda line: re.compile('(\D+) CC Plc Receive data has not changed for 45 sec is in a NORMAL state, Tagname = (\D+)').search(line) is None)
#             .filter(lambda line: 'HPU Main Accumulator Press Lo ALARM is in a NORMAL state, Tagname = HPU_MAINACC_PRESS_LO_ALM' not in line)
#         .filter(lambda line: 'HPU Main Accumulator Press Lo ALARM is in an ALARM state, Tagname = HPU_MAINACC_PRESS_LO_ALM' not in line)
#              .filter(lambda line: re.compile('UPS (A|B) LOSS OF POWER is Not in Alarm State, Tagname = ').search(line) is None)
#              .filter(lambda line: re.compile('UPS (A|B) LOSS OF POWER is in Alarm State, Tagname = ').search(line) is None)
           .filter(lambda line: re.compile('Hourly Log : (Yellow|Blue) Pod Leakage = (\d+).(\d+) gal').search(line) is None)
          .filter(lambda line: re.compile('(Yellow|Blue) (A|B) Flowmeter Flow Stopped. Total Flow = (\d+).(\d+) gal in (\d+) Seconds.').search(line) is None)
          .filter(lambda line: re.compile('(Yellow|Blue) (A|B) Flowmeter. Leakage = (\d+).(\d+) gal').search(line) is None)
            .filter(lambda line: re.compile('(Yellow|Blue) (A|B) Flowmeter leakage = (\d+).(\d+) gal; Cleared').search(line) is None)
             .take(10))
print rddparsed1

[]


In [140]:
rddparsed1 = (rdd.filter(lambda line: '=' not in line)
              .filter(lambda line: '>' not in line)
              .filter(lambda line: 'Multiple valve command from' not in line)
              .filter(lambda line: not line.endswith('      Lock or Unlock Commands in this Multiple Valve Command.'))
              .filter(lambda line: not line.endswith('      No Valve Commands in this Multiple Valve Command.'))
              
              
              .filter(lambda line: re.compile('E-PUMP-(\d) is (Running|Stopped)').search(line) is None)
              .filter(lambda line: re.compile('Lube Pump is (Running|Stopped)').search(line) is None)
              .filter(lambda line: re.compile('Glycol Pump is (Running|Stopped)').search(line) is None)
              
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) SEM is available.').search(line) is None)
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) SEM is NOT available.').search(line) is None)
              
              .filter(lambda line: not line.endswith('The BOP Test Box is Active'))
              .filter(lambda line: not line.endswith('The BOP Test Box is Not Active'))
              
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) Sem Commanded by: BOP Test Box').search(line) is None)
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) Sem Commanded by: DCP (A|B)').search(line) is None)
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) Sem Commanded by: Source address not received in time').search(line) is None)
              
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) Flowmeter Flow Started').search(line) is None)
              
              #Multi-line starts
              .filter(lambda line: re.compile('(Ya|Ba|Yb|Bb) (\D+) Received Auto SEM Regulate Command for Analog Channel (\d)').search(line) is None)
              .filter(lambda line: re.compile('     Set to (\d+) psi').search(line) is None)
              .filter(lambda line: re.compile('     Set to -(\d+) psi').search(line) is None)
              #Multi-line ends
              
              .filter(lambda line: re.compile('TCP (Left|Right) Enable (Up|Down)').search(line) is None)
              .filter(lambda line: re.compile('DCP (Left|Right) Enable (Up|Down)').search(line) is None)
              
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) Sem Commanded by: TCP (A|B)').search(line) is None)
              
              .filter(lambda line: re.compile('The (Tcp|Dcp) Run and Pull Mode is (Enabled|Disabled)').search(line) is None)
              
              .filter(lambda line: not line.endswith('No Soleniods with Bad DSDM Error Code.'))
              .filter(lambda line: re.compile('(\d+) Soleniods with Bad DSDM Error Code. See the Alarms screen, Show Faults and Mismatches.').search(line) is None)
              
              .filter(lambda line: re.compile('Heartbeat Loss, the (TCP|DCP) (A|B) HMI is Not Available').search(line) is None)
              .filter(lambda line: re.compile('Heartbeat Loss, the (TCP|DCP|HPU)  PLC is Not Available').search(line) is None)
              .filter(lambda line: re.compile('Heartbeat Ok, the (TCP|DCP) (A|B) HMI is Available').search(line) is None)
              .filter(lambda line: re.compile('Heartbeat Ok, the (TCP|DCP|HPU)  PLC is Available').search(line) is None)
              
              
              .filter(lambda line: re.compile('The Event Logger is (Shutting Down|Powering Up|is using WW Tag Names|using WW Tag Names)').search(line) is None)
              
              .filter(lambda line: re.compile('Remote Eds (is DISABLED|Trigger is OFF)').search(line) is None)
              
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) CC PLC is  generating the heartbeat.').search(line) is None)
              .filter(lambda line: re.compile('(Blue|Yellow) (A|B) CC PLC is not generating the heartbeat.').search(line) is None)
              
              .filter(lambda line: re.compile('(TCPA|DCPA|TCPB|DCPB) has (Initiated an|completed the) Align Sem').search(line) is None)
              
              .filter(lambda line: not line.endswith('Power On,  The Active Sem is Unknown'))
              
              
              .take(10))

print rddparsed1

[]


### 3. Parsing analog data and commands

#### 3.1 Analog data which has 2 dashes in the line

In [3]:
#Parsing lines with 2 dashes '-->'
rddparsed1 = (
    rdd.filter(lambda line: '-->' in line)
    .filter(lambda line: '--->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .map(lambda line: (extractTimeStamp.findall(line)[0], extractTimeStamp.split(line)[1].lstrip()))
    .map(lambda (timestamp, line): (str(line.split('-->')[0].rstrip().rsplit(' ',1)[0].replace(' ','_')),
                                    np.array([str2utc(timestamp),
                                              float(line.split('-->')[0].rstrip().rsplit(' ',1)[1]),
                                              float(line.split('-->')[1].lstrip().rsplit(' ',1)[0]),
                                              line.split('-->')[1].lstrip().rsplit(' ',1)[1]])))
    .reduceByKey(lambda accum, n: np.vstack((accum,n)))
    .map(lambda (var, array): (var, pd.DataFrame(data=array, columns=['AcqTime','From','To','Unit'])))
    .collect()
)

for (var, df) in rddparsed1:
    df.to_csv(os.path.join('ensco5006',var+'.csv'), index=False)
    
# print rddparsed1.map(lambda (var, df):var).collect()
# print rddparsed1.map(lambda (var, df): (var, df.head(5))).take(1)


#### 3.2 Check lines with 3 dashes, and see what data is in all the files. Check whether the states on left and right hand sides are numerical, which would be ananlog data, or string, which would be commands

In [8]:
#Check numerical data in lines with 3 dashes '--->' or more

#Numerical data on both sides
rddparsed2 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .take(5)
)
print 'rddpasred2'; print rddparsed2

#Numerical data on neither sides
rddparsed3 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: not isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .take(5)
)
print 'rddpasred3'; print rddparsed3

#Numerical data on From side only
rddparsed4 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .take(5)
)
print 'rddpasred4'; print rddparsed4


#Numerical data on From side only, and doesn't have "**ERROR** Valve Status = 123"
rddparsed42 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .filter(lambda line: '**ERROR** Valve Status = 123 --->' not in line)
    .take(5)
)
print 'rddpasred42'; print rddparsed42


#Numerical data on From side only, and doesn't have "**ERROR** Valve Status = 123", or "received a single valve command"
rddparsed43 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .filter(lambda line: '**ERROR** Valve Status = 123 --->' not in line)
    .filter(lambda line: 'received a single valve command' not in line)
    .take(5)
)
print 'rddpasred43'; print rddparsed43


#Numerical data on To side only, which doesn't exist
# rddparsed5 = (
#     rdd.filter(lambda line: '--->' in line)
#     .filter(lambda line: '---->' not in line)
#     .map(lambda line: re.sub(' +',' ',line))
#     .filter(lambda line: not isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
#     .filter(lambda line: isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
#     .take(5)
# )
# print 'rddparsed5'; print rddparsed5

#4 dashes
rddparsed6 = (
    rdd.filter(lambda line: '---->' in line)
    .filter(lambda line: '----->' not in line)
    .take(5)
)
print 'rddparsed6'; print rddparsed6

#4 dashes, but not in a multi-valve command
rddparsed62 = (
    rdd.filter(lambda line: '---->' in line)
    .filter(lambda line: '----->' not in line)
    .filter(lambda line: '       ' not in line)
    .take(5)
)
print 'rddparsed62'; print rddparsed62


#4 dashes, but not in a multi-valve command, or has "Active Sem Change"
rddparsed63 = (
    rdd.filter(lambda line: '---->' in line)
    .filter(lambda line: '----->' not in line)
    .filter(lambda line: '       ' not in line)
    .filter(lambda line: 'Active Sem Change' not in line)
    .take(5)
)
print 'rddparsed63'; print rddparsed63



#5 dashes, which doesn't exist
# rddparsed7 = (
#     rdd.filter(lambda line: '----->' in line)
#     .filter(lambda line: '------>' not in line)
#     .take(5)
# )
# print 'rddparsed7'; print rddparsed7


rddpasred2
[u'Mon Apr 04 12:15:12 2016 Blue A Stack Inclination X-Direction 1.36 ---> -0.64 degrees', u'Mon Apr 04 12:15:12 2016 Blue A SEM Temperature 102.92 ---> 32.00 DegF', u'Mon Apr 04 12:15:12 2016 Blue A SEM Humidity 16.30 ---> 0.00 Percent', u'Mon Apr 04 12:15:14 2016 Yellow A Stack Inclination X-Direction -0.26 ---> 0.88 degrees', u'Mon Apr 04 12:15:14 2016 Blue A SEM Temperature 32.00 ---> 89.78 DegF']
rddpasred3
[u'Fri Apr 01 22:03:37 2016 HPU MUD GAS SEPERATOR VALVE CLOSE ---> Fault', u'Sat Apr 02 00:17:20 2016 HPU TRIP TANK RETURN VALVE OPEN ---> Fault', u'Sat Apr 02 01:43:36 2016 HPU TRIP TANK FILL-IN VALVE CLOSE ---> Fault', u'Sat Apr 02 01:54:48 2016 HPU RESET RING TO DIV HSING LOCK ---> Fault', u'Sat Apr 02 03:42:13 2016 HPU DIVERTER LOCKDOWN DOGS UNLOCK ---> Fault']
rddpasred4
[u'Mon Apr 04 12:15:14 2016 SEM Blue A UPPER ANNULAR PREVENTER **ERROR** Valve Status = 123 ---> VENT', u'Mon Apr 04 12:15:14 2016 SEM Blue A LMRP CONNECTOR **ERROR** Valve Status = 123 ---> VEN

#### 3.3. Parse analog data lines containing 3 dashes


In [4]:
#Parsing lines with 3 dashes '--->' AND numerical data on both ends of dashes

rddparsed2 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .map(lambda line: (extractTimeStamp.findall(line)[0], extractTimeStamp.split(line)[1].lstrip()))
    .map(lambda (timestamp, line): (str(re.sub(' +', ' ', re.sub(r'[\W_]', ' ', line.split('--->')[0].rstrip().rsplit(' ',1)[0])).replace(' ','_')),
                                    np.array([str2utc(timestamp),
                                              float(line.split('--->')[0].rstrip().rsplit(' ',1)[1]),
                                              float(line.split('--->')[1].lstrip().rsplit(' ',1)[0]),
                                              line.split('--->')[1].lstrip().rsplit(' ',1)[1]])))
    .reduceByKey(lambda accum, n: np.vstack((accum,n)))
    .map(lambda (var, array): (var, pd.DataFrame(data=array, columns=['AcqTime','From','To','Unit'])))
    .collect()
)

for (var, df) in rddparsed2:
    df.to_csv(os.path.join('ensco5006',var+'.csv'), index=False)
    

#### 3.4 Parse command lines containing 3 dashes

##### find out states that are longer than 1 words

In [7]:
rddparsed3 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: not isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .map(lambda line: extractTimeStamp.split(line)[1].lstrip())
    .map(lambda line: line.split('--->')[1].lstrip())
    .filter(lambda line: len(line.rsplit(' ',1))>1)
    .countByValue()
    )

print rddparsed3

defaultdict(<type 'int'>, {u'VENT Sol 1 Lock Sol 2 Lock': 12, u'HP CLOSE': 71, u'LATCH Sol 1 Lock Sol 2 Lock': 4, u'OPEN Sol 1 Lock Sol 2 Lock': 10, u'Both Diags On': 764, u'VENT Lock': 8, u'CLOSE Break Lock': 1, u'**ERROR** Valve Status = 123': 1857, u'VENT Break': 4})


##### find out states that are exactly 1 word

In [8]:
rddparsed3 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: not isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .map(lambda line: extractTimeStamp.split(line)[1].lstrip())
    .map(lambda line: line.split('--->')[1].lstrip())
    .filter(lambda line: len(line.rsplit(' ',1))==1)
    .countByValue()
    )

print rddparsed3

defaultdict(<type 'int'>, {u'': 8, u'VENT': 5056, u'EXTEND': 206, u'DISARM': 763, u'YELLOW': 11, u'RETRACT': 141, u'DECREASE': 4084, u'cameron': 36, u'LATCH': 841, u'OPEN': 48512, u'BLOCK': 5855, u'DUMP': 322, u'ENERGIZE': 411, u'INCREASE': 4182, u'Drilling': 15, u'LOCK': 6306, u'CAMERON': 1, u'CLOSE': 46709, u'None': 2, u'UNLATCH': 1235, u'Fault': 17740, u'ISOLATE': 654, u'PRESSURE': 136, u'UNLOCK': 358, u'drilling': 39, u'RELEASE': 150, u'HOLD': 131, u'BLUE': 6, u'DE-ENERG': 322, u'CHARGE': 149, u'ARM': 350, u'DRILLING': 9})


##### knowing what possible states there are, parse the commands

In [15]:
def com_parse(lineleft):
    com_list = ['VENT Sol 1 Lock Sol 2 Lock', 'HP CLOSE', 'LATCH Sol 1 Lock Sol 2 Lock', 
            'OPEN Sol 1 Lock Sol 2 Lock', 'Both Diags On', 'VENT Lock', 
            'CLOSE Break Lock', '**ERROR** Valve Status = 123', 'VENT Break',
            'VENT', 'EXTEND', 'DISARM', 'YELLOW', 'RETRACT', 'DECREASE', 'cameron', 'LATCH', 
                'OPEN', 'BLOCK', 'DUMP', 'ENERGIZE', 'INCREASE', 'Drilling', 'LOCK',
                'CAMERON', 'CLOSE', 'None', 'UNLATCH', 'Fault', 'ISOLATE', 'PRESSURE', 
                'UNLOCK', 'drilling', 'RELEASE', 'HOLD', 'BLUE', 'DE-ENERG', 'CHARGE', 'ARM', 'DRILLING'
           ]
    if ':' in lineleft:
        com_out = lineleft.split(':')[0].rstrip()
        from_out = lineleft.split(':')[1].lstrip()
    else:
        lineleft = re.sub(' +', ' ', lineleft)
        if lineleft.endswith(tuple(com_list)):
            #some states could have the same ending characters, e.g. 'VENT Lock' and 'LOCK'
            #this if-else logic below is to make sure that the LONGEST state is used to parse a line
            lineend = [k for k in com_list if lineleft.endswith(k)]
            if len(lineend) == 1:
                from_out = lineend[0]
                com_out = lineleft[:-len(from_out)].rstrip()
            else:
                lineend_flag = [k.endswith(tuple([n for n in lineend if n != k])) for k in lineend]
                if sum(lineend_flag)==1:
                    from_out = [v for (k,v) in enumerate(lineend) if lineend_flag[k]==1][0]
                    com_out = lineleft.split(from_out)[0].rstrip()
                else:
                    com_out = lineleft
                    from_out = 'ParsingError'
        else:
            com_out = lineleft
            from_out = 'ParsingError'
            
    return com_out, from_out



rddparsed4 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .filter(lambda line: 'received a single valve command' not in line)
    .filter(lambda line: 'was sent a single valve command from' not in line)
    .map(lambda line: re.sub(' +', ' ', line))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .map(lambda line: (extractTimeStamp.findall(line)[0], extractTimeStamp.split(line)[1].lstrip()))
    .map(lambda (timestamp, line): (str2utc(timestamp), line.split('--->')[0].rstrip(), line.split('--->')[1].lstrip()))
    .map(lambda (acqtime, lineleft, lineright): (acqtime, com_parse(lineleft)[0], com_parse(lineleft)[1], lineright))
#     .filter(lambda (acqtime, com_out, from_out, to_out): from_out == 'ParsingError')
    
#     .map(lambda (acqtime, com_out, from_out, to_out): (com_out.replace(' ','_'), [acqtime, from_out, to_out]))
#     .reduceByKey(lambda accum, n: np.vstack((accum,n)))
#     .map(lambda (var, array): (var, pd.DataFrame(data=array, columns=['AcqTime','From','To'])))
#     .takeSample(withReplacement=False, num=5)
    
    .map(lambda (acqtime, com_out, from_out, to_out): ([acqtime, com_out.replace(' ','_'), from_out, to_out]))
    .collect()
)
df4 = pd.DataFrame(data=rddparsed4, columns=['AcqTime','Command','From','To'])
print df4.head(10)
print len(df4)
df4.to_csv(os.path.join('ensco5006','commands.csv'), index=False)



      AcqTime                        Command    From      To
0  1459566217    HPU_MUD_GAS_SEPERATOR_VALVE   CLOSE   Fault
1  1459574240     HPU_TRIP_TANK_RETURN_VALVE    OPEN   Fault
2  1459579416    HPU_TRIP_TANK_FILL-IN_VALVE   CLOSE   Fault
3  1459580088    HPU_RESET_RING_TO_DIV_HSING    LOCK   Fault
4  1459586533     HPU_DIVERTER_LOCKDOWN_DOGS  UNLOCK   Fault
5  1459586846  HPU_STARBOARD_OVERBOARD_VALVE    OPEN   Fault
6  1459590838       HPU_PORT_OVERBOARD_VALVE   CLOSE   Fault
7  1459591342       HPU_PORT_OVERBOARD_VALVE   Fault   CLOSE
8  1459591345  HPU_STARBOARD_OVERBOARD_VALVE   Fault    OPEN
9  1459591345     HPU_DIVERTER_LOCKDOWN_DOGS   Fault  UNLOCK
98041
        AcqTime  Destination        Origin                   Equipment  \
0  1459793034.0  Yellow B CC  BOP Test Box   YELLOW LMRP STINGER SEALS   
1  1459793036.0  Yellow A CC  BOP Test Box   YELLOW LMRP STINGER SEALS   
2  1459793049.0  Yellow A CC  BOP Test Box         YELLOW LMRP STINGER   
3  1459793049.0  Yellow B C

##### Parse the lines that uses "received" or "sent" in a line, which indicates issuing of the command

In [ ]:
# Examples:
# 'Mon Apr 04 15:45:33 2016 Yellow A CC received a single valve command from BOP Test Box: RAM 1 ---> CLOSE'
# 'Sun Apr 10 07:03:09 2016  Hpu was sent a single valve command from  TCP B: TRIP TANK DISCHARGE VALVE ---> CLOSE'
rddparsed42 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .filter(lambda line: 'received a single valve command' in line)
    
    .map(lambda line: re.sub(' +', ' ', line))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .map(lambda line: (extractTimeStamp.findall(line)[0], extractTimeStamp.split(line)[1].lstrip()))
    .map(lambda (timestamp, line): (str2utc(timestamp), line.split('--->')[0].rstrip(), line.split('--->')[1].lstrip()))
    .map(lambda (acqtime, lineleft, lineright): [acqtime, 
                                                 lineleft.split('received')[0].rstrip(), 
                                                 lineleft.split('from')[1].lstrip().split(':')[0],
                                                 lineleft.split(':')[1].lstrip(), 
                                                 lineright])
     .reduce(lambda x, y: np.vstack((x,y)))
    )
df42 = pd.DataFrame(data=rddparsed42, columns=['AcqTime','Destination','Origin', 'Equipment', 'To'])

rddparsed43 = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .filter(lambda line: 'was sent a single valve command from' in line)
    
    .map(lambda line: re.sub(' +', ' ', line))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .map(lambda line: (extractTimeStamp.findall(line)[0], extractTimeStamp.split(line)[1].lstrip()))
    .map(lambda (timestamp, line): (str2utc(timestamp), line.split('--->')[0].rstrip(), line.split('--->')[1].lstrip()))
    .map(lambda (acqtime, lineleft, lineright): [acqtime, 
                                                 lineleft.split('was sent')[0].rstrip(), 
                                                 lineleft.split('from')[1].lstrip().split(':')[0],
                                                 lineleft.split(':')[1].lstrip(), 
                                                 lineright])
     .reduce(lambda x, y: np.vstack((x,y)))
    )
df43 = pd.DataFrame(data=rddparsed43, columns=['AcqTime','Destination','Origin', 'Equipment', 'To'])

df44 = pd.concat([df42, df43])

print df44.head(10)
print len(df44)
df44.to_csv(os.path.join('ensco5006','command_ack.csv'), index=False)


#### 3.5 Multiple valve commands, which starts with a line that has a timestamp and appends lines without a timestamp

In [51]:
def mvc_lines(data):
    mvc_start = [(k,line) for (k,line) in enumerate(data) if 'Multiple valve command from' in line]
    
    mvc = []
    for (k,line) in mvc_start:
#         'Mon Apr 04 13:03:53 2016 Multiple valve command from  BOP Test Box to Yellow A '
#         'Mon Apr 04 13:04:52 2016 Multiple valve command from  BOP Test Box to Yellow A '
        acqtime = str2utc(extractTimeStamp.findall(line)[0])
        lineleft = extractTimeStamp.split(line)[1].lstrip()
        destination = lineleft.split('to')[1].lstrip().rstrip()
        origin = lineleft.split('from')[1].split('to')[0].lstrip().rstrip()
        
        j = k+1
        mvc_command = []
        while j>k:
            if len(extractTimeStamp.findall(data[j]))==0:
                command = data[j].lstrip()
                mvc_command += [command]
                j+=1
            else:
                break
        if len(mvc)==0:
            mvc = np.array([acqtime, destination, origin, mvc_command], dtype=object)
        else:
            mvc = np.vstack((mvc, np.array([acqtime, destination, origin, mvc_command], dtype=object)))
        
    return mvc


rddwhole = sc.wholeTextFiles('ensco5006/*.log')
rddmvc = (rddwhole
          .filter(lambda (fn, data): 'Multiple valve command from' in data)
          .map(lambda (fn, data): (fn, data.split('\r\n')))
          .map(lambda (fn, data): mvc_lines(data))
          .reduce(lambda x, y: np.vstack((x,y)))
          )
print 'rddmvc'
dfmvc = pd.DataFrame(data=rddmvc, columns=['AcqTime','Destination','Origin', 'MVC'])
dfmvc.to_csv(os.path.join('ensco5006','mvc.csv'), index=False)

# rddmvc1 = (rddwhole
#           .filter(lambda (fn, data): 'Multiple valve command from' in data)
#           .map(lambda (fn, data): (fn, data.split('\r\n')))
#           .map(lambda (fn, data): (fn, mvc_lines(data)))
#           .filter(lambda (fn, lines): '      Lock or Unlock Commands in this Multiple Valve Command.' in lines)
#           .filter(lambda (fn, lines): '      No Valve Commands in this Multiple Valve Command.' in lines)
#           .map(lambda (fn, lines): lines)
#           .collect()
#          )
# print'rddmvc1'
# print len(rddmvc1)


rddmvc


#### 3.6 Parse lines that indicate when active SEM is changed

In [56]:
rddparsed63 = (
    rdd.filter(lambda line: '---->' in line)
    .filter(lambda line: '----->' not in line)
    .filter(lambda line: '       ' not in line)
    .map(lambda line: re.sub(' +', ' ', line))
    .map(lambda line: (extractTimeStamp.findall(line)[0], extractTimeStamp.split(line)[1].lstrip()))
    .map(lambda (timestamp, line): (str2utc(timestamp), line.split('---->')[0].rstrip(), line.split('--->')[1].lstrip()))
    .map(lambda (acqtime, lineleft, lineright): (acqtime, lineleft.split(':')[1].lstrip(), lineright))
    .collect()
)

df_semchange = pd.DataFrame(data=rddparsed63, columns=['AcqTime', 'From', 'To'])
df_semchange.to_csv(os.path.join('ensco5006','active_sem_change.csv'), index=False)


### 4. Check if all lines are parsed or at least "covered" by a regex

In [ ]:
rdd = sc.textFile('ensco5006/*.log',100)
rdd.persist()

rddtotal = rdd.count()


def mvc_lines_new(data):
    mvc_start = [(k,line) for (k,line) in enumerate(data) if 'Multiple valve command from' in line]
    
    mvc = []
    for (k,line) in mvc_start:
        mvc += [line]
        j = k+1
        while j>k:
            if len(extractTimeStamp.findall(data[j]))==0:
                command = data[j].lstrip()
                mvc += [command]
                j+=1
            else:
                break
    return mvc


rddparsed1_count = (
    rdd.filter(lambda line: '-->' in line)
    .filter(lambda line: '--->' not in line)
    .count()
    )

rddparsed2_count = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +',' ',line))
    .filter(lambda line: isdigit(line.split('--->')[0].rstrip().rsplit(' ',1)[1]))
    .filter(lambda line: isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .count())

rddparsed4_count = (
    rdd.filter(lambda line: '--->' in line)
    .filter(lambda line: '---->' not in line)
    .map(lambda line: re.sub(' +', ' ', line))
    .filter(lambda line: not isdigit(line.split('--->')[1].lstrip().rsplit(' ',1)[0]))
    .count())
    
    
rddparsed63_count = (
    rdd.filter(lambda line: '---->' in line)
    .filter(lambda line: '----->' not in line)
    .filter(lambda line: '       ' not in line)
    .count())
    

rddparsed1_count1 = (rdd.filter(lambda line: '>' in line)
                      .filter(lambda line: '->' not in line)
                     .count())

rddparsed1_count2 = (rdd.filter(lambda line: '=' in line)
                     .filter(lambda line: '<' not in line)
                     .filter(lambda line: '>' not in line)
                     .count()
                     )

rddparsed1_count3 = (rdd.filter(lambda line: '=' not in line)
                     .filter(lambda line: '>' not in line)
                     .filter(lambda line: 'Multiple valve command from' not in line)
                  .filter(lambda line: '      ' not in line)
                     .count()
                     )

rddparsed1_count4 = rdd.filter(lambda line: '<' in line).count()

mvc_count = (sc.wholeTextFiles('ensco5006/*.log')
             .filter(lambda (fn, data): 'Multiple valve command from' in data)
          .map(lambda (fn, data): (fn, data.split('\r\n')))
          .map(lambda (fn, data): (fn, mvc_lines_new(data)))
          .map(lambda (fn, lines): (fn, len(lines)))
          .map(lambda (fn, no_lines): no_lines)
         .reduce(lambda x, y: x+y))


parsedtotal = rddparsed1_count+rddparsed2_count+mvc_count+rddparsed4_count+rddparsed63_count
parsedtotal = (rddparsed1_count+rddparsed2_count+mvc_count+rddparsed4_count+rddparsed63_count+
               rddparsed1_count1+rddparsed1_count2+rddparsed1_count3+rddparsed1_count4)

print "{0} out of {1} parsed".format(parsedtotal, rddtotal)
print "{0:.5f}% parsed".format(parsedtotal/float(rddtotal)*100)


In [ ]:
parsedtotal = rddparsed1_count+rddparsed2_count+mvc_count+rddparsed4_count+rddparsed63_count
print parsedtotal
print rddtotal